In [ ]:
import bgk
import matplotlib.pyplot as plt
import bgk.run_params as rp
import os

In [ ]:
# Parameters to adjust figures
Bs = [.5, 1, 2]
ress = [256, 512]
case = ["exact", "max", "exact-rev", "max-rev"][0]

nframes = 100
param = rp.ne

In [ ]:
path_fmt = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/{case}/B{{:05.2f}}-n{{}}/"

def get_line(B, res, nframes, param) -> tuple[list[float], list[float]]:
    path = path_fmt.format(B, res)

    loader = bgk.Loader(path, engine="pscadios2", species_names=['e', 'i'])
    videoMaker = bgk.VideoMaker(nframes, loader)

    wholeSlice = bgk.DataSlice(slice(None, None), "")
    videoMaker.loadData(param)
    videoMaker.setSlice(wholeSlice)
    
    return videoMaker.times, videoMaker._getMeansAtOrigin(res//128)

In [ ]:
# Load Data
liness: list[list[tuple]] = []
for B in Bs:
    liness.append([])
    for res in ress:
        print("Loading " + path_fmt.format(B, res) + "...")
        x, y = get_line(B, res, nframes, param)
        liness[-1].append((x, y))

In [ ]:
# Generate Figure
fig, ax = plt.subplots()

ax.set_xlabel("Time")
ax.set_ylabel(f"Mean {param.title}")
ax.set_title(f"Comparisons of Mean {param.title} Near Origin")

for B, lines in zip(Bs, liness):
    for res, (x, y) in zip(ress, lines):
        ax.plot(x, y, label=f"$B_0={B}, n={res}$")


ax.legend();

In [ ]:
def get_fig_path(outdir, fig_name: str) -> str:
    return os.path.join(outdir, fig_name)

def get_fig_name() -> str:
    param_str = param.title.replace("_", "").replace("$", "")
    Bs_str = ",".join([f"B{B:05.2f}" for B in Bs])
    return f"scaling-{param_str}-{case}-{Bs_str}.png"

In [ ]:
# Save Figure
outdir = f"/mnt/lustre/IAM851/jm1667/psc-scrap/figs-{case}/"
os.makedirs(outdir, exist_ok=True)

fig.savefig(get_fig_path(outdir, get_fig_name()), bbox_inches="tight", pad_inches=0.01, dpi=300)